# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
weather_data_path = "output_data/cities.csv"
weather_data = pd.read_csv(weather_data_path)
weather_data.head()

,Unnamed: 0,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Windspeed,Country
0,0,vila,42.03,-8.16,62.996,87.0,0.0,3.10,PT
1,1,rikitea,-23.12,-134.97,67.262,59.0,44.0,6.20,PF
2,2,batticaloa,7.71,81.69,81.572,76.0,87.0,1.38,LK
3,3,hervey bay,-25.30,152.85,64.004,95.0,71.0,2.68,AU
4,4,attawapiskat,0.00,0.00,0.000,0.0,0.0,0.00,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_data["Humidity"].astype(float)

In [23]:
# Plot Heatmap
heat_fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# My ideal weather conditions:
min_temp = weather_data[weather_data["Max_Temp"]>50]
max_temp = min_temp[min_temp["Max_Temp"]<80]
wind_speed = max_temp[max_temp["Windspeed"]<7]
cloudiness = wind_speed[wind_speed["Cloudiness"]<75]
my_vacation = cloudiness[cloudiness["Humidity"]<75].reset_index(drop=True)
my_vacation

,Unnamed: 0,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Windspeed,Country
0,1,rikitea,-23.12,-134.97,67.262,59.0,44.0,6.20,PF
1,5,ribeira grande,38.52,-28.70,69.404,71.0,13.0,2.60,PT
2,7,busselton,-33.65,115.33,57.002,74.0,41.0,3.99,AU
3,16,port alfred,-33.59,26.89,62.204,35.0,0.0,3.88,ZA
4,22,saint paul,44.94,-93.09,77.000,61.0,1.0,3.60,US
...,...,...,...,...,...,...,...,...,...
94,563,pachino,36.72,15.09,75.992,64.0,0.0,1.50,IT
95,583,bogandinskiy,56.89,65.89,57.200,71.0,2.0,4.00,RU
96,591,taltal,-25.40,-70.48,50.738,60.0,0.0,2.34,CL
97,593,bourges,47.25,2.42,64.004,59.0,12.0,1.50,FR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
my_vacation['Hotel Name'] = ""
hotel_df = pd.DataFrame(my_vacation)
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}

In [26]:
for index, row in hotel_df.iterrows():
    
    params["location"] =  f'{row["Latitude"]},{row["Longitude"]}'

    # get restaurant type from df
    hotel_location = row['City']


    # assemble url and make API request
    print(f"Retrieving Results for: {hotel_location}.")
    response = requests.get(base_url, params=params).json()
    print(response)
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['Name']

                  
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for: rikitea.
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.1206197, 'lng': -134.9687529}, 'viewport': {'northeast': {'lat': -23.1194598197085, 'lng': -134.9673886697085}, 'southwest': {'lat': -23.1221577802915, 'lng': -134.9700866302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '60864d02c84a997a4e41d2ee16ce4b24b3e67a6a', 'name': "Pension Maro'i", 'opening_hours': {'open_now': True}, 'photos': [{'height': 2784, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111098332724302207930">luigia b.</a>'], 'photo_reference': 'CmRaAAAA6DdRG5dF69vb5FVESVK8inutg00om4Se7mjlwPGbX9qyZaDces-Fnz4WBhi0depi-1EwEFAxiTmkIdWkfQYzVk2Ts2i_ke_dCsJC5Jk69nwYBLILm1dsZRUCdbOiEJ2_EhAgHoqaR4-cTJiL8GQkdosRGhSWvBgJLN2jdm5wvYUPdEHjMt5dag', 'width': 1856}], 'place_id': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw', 'plus_code': {'compound_code': 'V2HJ+QF Rikitea, French Polynesia', 'global_

{'html_attributions': [], 'next_page_token': 'CqQCHgEAADx17zcowzUAAAKAOWvx-jau4o7S3wZ-P-_mfIIj2zl9eFCZy1fKwNeEjA9Ee3u6RCLwMpEqLFaafCdNirFoK9gE1Iu453TtWJwCtqsGyq-zzzBmARi9qMvtj_od4ANhYkiUx5GuBWJob8Hs8bhJMckQzRc6dhnokY4sjj5DF708fHGmY0f37j_emBfX4lN47RKWQOjQA_fqhVdRRYmhD-A7WxdCRbg5JT0Xhxn0g4QLhD0Q6KPuOjFb6ixo01ED0RLdXq3pPZCubK_3XwGiX0JQ3xLG_JKuxgsX_nBQa_UfU00cBnki_72yUWhjsC_URkQ7UFwTRkJ33mWNQMByotcmbw3QxEuEYKh7eIKntazYz7kY4mZMY-UF6D_GkZn2JhIQezsHyr0mG1VKvhtVOgbBPRoU9JTLdInQksksWNMiNCwYikqTpb8', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -33.6517571, 'lng': 115.3251415}, 'viewport': {'northeast': {'lat': -33.6505384197085, 'lng': 115.3265634302915}, 'southwest': {'lat': -33.6532363802915, 'lng': 115.3238654697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '24d2ed094bca49273129bef54ec2414d891758f1', 'name': 'Busselton Beach House', 'photos': [{'height': 1281, 'html_attributions': ['<a href="https://maps.google.co

{'html_attributions': [], 'next_page_token': 'CqQCHgEAACw50XogdCA3odq1hOqhitYQWC4zbYD8bOYy9wJYLkLEqBG6K9ghaEylKSbcZcXqX7Na_r5GklA-DcrNiA4tEuM4msa7upqxCwQeTfTp4o8YlMWx2z7xc2AIbXrhwA9pGQKccbMwZUnatnAEKsTFEsrD9BFdgOqxBPBXf7zTBbMgyFemXqJpCvlCsgR-m1G5r4wdeXcKNBPIKoDBfGzwL0EZq02A4err1sdPtHNDbYUsLzQmlXUFeu3LGCGcpnqPXJO1DWcJM3NSAciAvROG2_M8hvU3OVt58t4DHr-vQx_n_rcRu7Kf4k7O04D2fbhxkNFajaAeVHVSwjBCqJvW7WgildCFo19s_FAw3fTdIPruHZCNzUHRWuaB9hYuC7473RIQuYuQ6nZQwaZxzLdpp4ritxoUBG0UmNOjTQdMKJ9W73L5wyxCEEw', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -33.5920597, 'lng': 26.8926044}, 'viewport': {'northeast': {'lat': -33.5906461697085, 'lng': 26.8939202802915}, 'southwest': {'lat': -33.5933441302915, 'lng': 26.8912223197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '1cccc61004f267c37ea48a45859e6b04d95f61d1', 'name': 'Dockside House', 'photos': [{'height': 2448, 'html_attributions': ['<a href="https://maps.google.com/maps/con

{'html_attributions': [], 'next_page_token': 'CqQCHgEAAAvpOneMVZVcZgSc2beX0xx5hd-gBqB82XsWPQ9I0E5_SC3i5HIuS1R4DMZuf5_CQPZlOKdVp9A-8XE4BnY-XUaytbZywH9vesN3Vfnw6TYihrAn74eWDBRI6ke7rN-b9lmELX9JWIQeI8OPveozyB44myqJlK_tcG35E2qMrHTdwtJY84q63goBEQirjqJ9WJ9r7AZnKsRLSz5wwrCqZDqRDxAJFTp55Bo7ireoCynzlQwiPn4olDC7FZHA1tQaLSV3fZJ4_7KD48LMBdWfuziXB36Urju_leetWEvIBXT4tGG0_UxpMbogxkkVwEaUiBvix_d4Q5Dq7rGVBUSyROzvCQ8adoYK_3B5H8aJ0TazreVbCOxTmTywZO3AQ5jw_xIQ5aU93jdZREOr0fFZDpaXShoUIPSHAPlxLSsgfz0POHu_Ow_ecIc', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 44.9403083, 'lng': -93.08885}, 'viewport': {'northeast': {'lat': 44.9414340302915, 'lng': -93.0873047697085}, 'southwest': {'lat': 44.9387360697085, 'lng': -93.0900027302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '4bd9a4463769769ea78552dfc05429c37c8f8203', 'name': 'West Side Flats', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJMeefCUfV94cRL7b3oYgS9AU', 'plus_code': 

{'html_attributions': [], 'next_page_token': 'CqQCHgEAAOQWL44fXgz_a47dO_gj81QGHylJkqhz2W0C8VkPY62qStTrwvb-79H510X7RCXwSEIgm2D4qTn0KkDTtWnS9PHtD2JZfPt0luGztTlecm5rRPjBMqyl3DWtjXpqJPeHZOsG7bYKADniyMFfr8btUQPDYgmVVUFenrOF0ZG-jLXA5COkHBFH-hwFvP7i96HOivZOoGqy7KPGgcEKpdMDwRZiFRlZFQTd2HyRqINRT1N3EO5tBIB8PhkviRpxnzUPnIsDbDbPKDhwTxdGx36iYCo-BlJ2VkgiNmLqu6aMkzhFVGlH-xcRGH6bPhRPzweLVEoQ1gcMNtqGM3huddu5gYNTpqocaZk0Eke-andOGyTSjkkR0pYrhlua8oYsWr76dRIQYYeIHSWwAc4xZxZEg-TvqBoUnO5nSm6rWxRaZBXJggyXOL3J8a4', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.7376386, 'lng': 116.8419976}, 'viewport': {'northeast': {'lat': -20.7362986697085, 'lng': 116.8431247802915}, 'southwest': {'lat': -20.7389966302915, 'lng': 116.8404268197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '3c7c17ee5e8ccd7d180a9fc244f33bd43e29af41', 'name': 'Karratha International Hotel', 'photos': [{'height': 2736, 'html_attributions': ['<a href="https://maps.go

...

{'html_attributions': [], 'next_page_token': 'CqQCHgEAAIeVpN1pIyzjWfcxU6REQ5gIALQYHbjJZYAi_X_lpWJz9BRHT7OfhB88DJNDr3noY7b5DlAHwVRM-NisrTgzZ1nVaijkva8ABwvQROxD_O2l54WuK1qc6p3ovJcFCTu6-ICqMXAnALkGEkxYmYroIS21wraA8TJEoTjTCwNuaHmYDfRSnz6U9Zl6YhjWRBwJhimEhpuP_xovc-nZF1KUk-d0m1QWdJV7i516srGNAi-NBT3nakFv1c7nQTil46cFYC4BWZTKq8oY9EWFBdOLmQUMgQMijJDsin4PnMoW6sWBi3s-Ah4IgjYAoS-kSnwftQfCKNf5V5Jm4Mz-_wp_pquulpIA8qm3nKy4TFJMTn5k1WRxUaGBHsJLAI8HA38WyRIQM2rd9fFoh8KFFbHFrtAS7hoU6d0bI_NU8_0jjsXKn0LOc3C4EcI', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 45.52008000000001, 'lng': -122.67998}, 'viewport': {'northeast': {'lat': 45.52147928029149, 'lng': -122.6786052697085}, 'southwest': {'lat': 45.5187813197085, 'lng': -122.6813032302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '9c5b10c39acf1f432c2338f28c5bc9e6b4054459', 'name': 'Downtown Luxury Suites by Barsala', 'place_id': 'ChIJx1M1iwQKlVQRuh6sFz6qLSU', 'plus_code': {'compou

{'html_attributions': [], 'next_page_token': 'CqQCHgEAADLiCJ0_9SXG9oFERgzE70-Oe54EG8MozHyn8Rl641TjGzccFWVHxEBh-HHPz5kA9VH0FuydmpEPOQm9goSN9bPlq8tmKxdOjsnIlEA13_-BJ0IeC0AkqvjL-hu9sCIPHHz-LM1ueAAPrkQ_l7LeCUldLcKY4IWtbEE7bDPzZoCL2FAgj3635e79zvUEpN9tlTleo-MZScUDDP_OxuwaWmpOZ7s1CxXWiPUUmaC1mcVL5YWNOoy8i3Vk9aHrsqVaJL-0XSV7hulGMUUsrjzyqNWh2EKa_izrwZ_VWLIQObXBOZkm85vJXZJsyTI5_MB52JC5NEJur-pNqykBqP4TD_ujszjsLZ7a_glB6HIrhxXC4HjfWfbCjPPvcTXP2nxJaxIQh9gv5PMYjXamrYNzU0zR8RoUxzODPCp_e96ltxmDVmb9T-R--r0', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 36.7195041, 'lng': 15.08909849999999}, 'viewport': {'northeast': {'lat': 36.7208383802915, 'lng': 15.0904070302915}, 'southwest': {'lat': 36.7181404197085, 'lng': 15.08770906970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'd3a6f4daaa8870cab2c8648951ececbdd483ebfa', 'name': 'Casa Vacanze Maria', 'place_id': 'ChIJrYiMwAUhEhMRy_QAC5em7rg', 'plus_code': {'compound_code': 'P39Q+RJ 

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 56.8891113, 'lng': 65.8737766}, 'viewport': {'northeast': {'lat': 56.8904405302915, 'lng': 65.87526158029151}, 'southwest': {'lat': 56.8877425697085, 'lng': 65.87256361970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/camping-71.png', 'id': 'ef4541b6cd30643aedeb8fe12a3a36365855f9b1', 'name': 'Alex', 'photos': [{'height': 800, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101048103319668235330">Jenya Green</a>'], 'photo_reference': 'CmRaAAAAg1EDg9B9VtvoXCWp2jxlOCtrwvnkKEC1OjGFHyIoPVNO4FokOyYxrXJ7D-yD1Av0hR637JnLVbCUmOpkG-gNPKQ7_OZUxgjK0maUDhAitUf7G2ja917f29O5-oIKuMKCEhCZZBmFgPIAFpFrz3ZuiuzyGhTDt7JJlnoZcZc27ED6BFlnUHuqrQ', 'width': 480}], 'place_id': 'ChIJ30IsMybRu0MR_kMp5OPm3bc', 'plus_code': {'compound_code': 'VVQF+JG Bogandinskiy, Tyumen Oblast, Russia', 'global_code': '9J87VVQF+JG'}, 'rating': 4, 'reference': 'ChIJ30IsMybRu0MR_kMp5OPm3bc

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 47.2749148, 'lng': 2.4471831}, 'viewport': {'northeast': {'lat': 47.2762658302915, 'lng': 2.448521730291501}, 'southwest': {'lat': 47.2735678697085, 'lng': 2.445823769708498}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '20fcb2026fc6ee192d5a41fbdeb0ab972235ffc0', 'name': 'Thomas', 'place_id': 'ChIJdeoedy2x-kcRp57iyENxCq0', 'plus_code': {'compound_code': '7CFW+XV Achères, France', 'global_code': '8FV47CFW+XV'}, 'rating': 3.5, 'reference': 'ChIJdeoedy2x-kcRp57iyENxCq0', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 2, 'vicinity': '160-210 Les Plures, Achères'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 47.2825026, 'lng': 2.442323}, 'viewport': {'northeast': {'lat': 47.28385523029149, 'lng': 2.443675730291502}, 'southwest': {'lat': 47.2811572697085, 'lng': 2.4409777

{'html_attributions': [], 'next_page_token': 'CqQCHgEAADcX0cUAgCfDsFh_bmLjcvELulWRAkn3UWaKoimiWQJiNIN-AxY0uvjsvVd5pb57XIURg2DHk0iyptDSt4f_CmD2MfRfhjGPVcM6Wc87pgXXcGXghalaGDu7pUP_Ko2P0aRqsqf8PrUUWAm_u6Q1ewmFxis0v2LRRBQQIAmMmHQTiKiZWIH7zYm8ii7yO3A9lNRDREtaX6DZihdnds4C8TQ8-FjccPL9x2XQ4ULcsu65IsrJYYDN2G4Mwh_XgSJKXm_HqjNmM25RxqomyEx-f1_9_BxUVi7zzohSSsIbqb7iqY2cYkp5wSogbhA-QOjNbvcqswlxohP-y_4JSqeSH7GS2MqCajLKTi3jAdtnuijF9eYiDWyCwqJRHgtogHKK7RIQKkRfJ5sO4Tk71h95PgsrrxoU_zcnyedU1g1MkLn_aTTo9Krb7x0', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -34.04860900000001, 'lng': 23.371046}, 'viewport': {'northeast': {'lat': -34.0472891697085, 'lng': 23.3723641802915}, 'southwest': {'lat': -34.0499871302915, 'lng': 23.3696662197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '62592ea1c85e59f35271b1799d97b023cf95b7f9', 'name': 'Beach House Plett', 'photos': [{'height': 4032, 'html_attributions': ['<a href="https://maps.google.com

In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [34]:
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))